# **Data Extraction**

In [ ]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# Function to extract text from url's
def extract_article_text(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Assuming the main article text is within <div class="td-post-content">
        article_div = soup.find('div', class_='td-post-content')
        if article_div:
            paragraphs = article_div.find_all('p')
            article_text = '\n'.join([p.text.strip() for p in paragraphs])
            article_title = soup.find('h1').text.strip()
            return article_title, article_text
        else:
            # Return default values if article text extraction fails
            return None, None
    else:
        return None, None


In [ ]:
# Read the Excel file
df = pd.read_excel('Input.xlsx')

In [ ]:
# Create a directory to store the extracted article texts
output_directory = 'EXTRACTED_ARTICLES'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
# Loop through each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    try:
        article_title, article_text = extract_article_text(url)

        # Save the extracted article as a text file
        file_path = os.path.join(output_directory, f'{url_id}.txt')
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(f'{article_title}\n\n{article_text}')

        print(f"Successfully extracted article for URL_ID: {url_id}")
    except Exception as e:
        print(f"Failed to extract article for URL_ID: {url_id}")
        print(f"Error: {str(e)}")


Successfully extracted article for URL_ID: 37
Successfully extracted article for URL_ID: 38
Successfully extracted article for URL_ID: 39
Successfully extracted article for URL_ID: 40
Successfully extracted article for URL_ID: 41
Successfully extracted article for URL_ID: 42
Successfully extracted article for URL_ID: 43
Successfully extracted article for URL_ID: 44
Successfully extracted article for URL_ID: 45
Successfully extracted article for URL_ID: 46
Successfully extracted article for URL_ID: 47
Successfully extracted article for URL_ID: 48
Successfully extracted article for URL_ID: 49
Successfully extracted article for URL_ID: 50
Successfully extracted article for URL_ID: 51
Successfully extracted article for URL_ID: 52
Successfully extracted article for URL_ID: 53
Successfully extracted article for URL_ID: 54
Successfully extracted article for URL_ID: 55
Successfully extracted article for URL_ID: 56
Successfully extracted article for URL_ID: 57
Successfully extracted article for

## Saving resullt in excel file for further text analysis

In [ ]:
# Read the Excel file containing the URLs
df = pd.read_excel('Input.xlsx')

# Create new empty columns for TITLE and TEXT
df['TITLE'] = ""
df['TEXT'] = ""

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    # Update the article_class with the correct class name for each website   article_class = 'entry-content'

    try:
        article_title, article_text = extract_article_text(url)

        # Update the DataFrame with the extracted article title and text

        df.at[index, 'TITLE'] = article_title
        df.at[index, 'TEXT'] = article_text


        print(f"Successfully extracted article for URL_ID: {url_id}")
    except Exception as e:
        print(f"Failed to extract article for URL_ID: {url_id}")
        print(f"Error: {str(e)}")

# Save the DataFrame with the extracted data back to the Excel file
df.to_excel('extracted_text.xlsx', index=False)

Successfully extracted article for URL_ID: 37
Successfully extracted article for URL_ID: 38
Successfully extracted article for URL_ID: 39
Successfully extracted article for URL_ID: 40
Successfully extracted article for URL_ID: 41
Successfully extracted article for URL_ID: 42
Successfully extracted article for URL_ID: 43
Successfully extracted article for URL_ID: 44
Successfully extracted article for URL_ID: 45
Successfully extracted article for URL_ID: 46
Successfully extracted article for URL_ID: 47
Successfully extracted article for URL_ID: 48
Successfully extracted article for URL_ID: 49
Successfully extracted article for URL_ID: 50
Successfully extracted article for URL_ID: 51
Successfully extracted article for URL_ID: 52
Successfully extracted article for URL_ID: 53
Successfully extracted article for URL_ID: 54
Successfully extracted article for URL_ID: 55
Successfully extracted article for URL_ID: 56
Successfully extracted article for URL_ID: 57
Successfully extracted article for

# **TEXT ANALYSIS**

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import re

In [ ]:
# Download nltk data (if not already downloaded)
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

# Load the input data from the "extracted_text.xlsx" file
input_file = "extracted_text.xlsx"
df = pd.read_excel(input_file)
df.head()

,URL_ID,URL,TITLE,TEXT
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL_ID  114 non-null    int64 
 1   URL     114 non-null    object
 2   TITLE   111 non-null    object
 3   TEXT    111 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.7+ KB


In [ ]:

# Function to clean text and tokenize
def clean_and_tokenize(text):
    if pd.notnull(text):  # Check if text is not null
        stop_words = set(stopwords.words("english"))
        words = word_tokenize(text.lower())
        return [word for word in words if word.isalpha() and word not in stop_words]
    else:
        return []  # Return an empty list if the text is null

# Clean the text and tokenize it
df["CLEANED_TEXT"] = df["TEXT"].apply(lambda x: clean_and_tokenize(x))

In [ ]:
# Load positive and negative words
with open('positive-words.txt', 'r', encoding='latin-1') as f:
    positive_words = set(f.read().splitlines())

with open('negative-words.txt', 'r', encoding='latin-1') as f:
    negative_words = set(f.read().splitlines())


In [ ]:
# Function to calculate positive and negative scores
def calculate_scores(tokens):
    pos_score = sum(1 for word in tokens if word in positive_words)
    neg_score = sum(1 for word in tokens if word in negative_words)
    return pos_score, neg_score

# Calculate Positive and Negative Scores
df[["POSITIVE_SCORE", "NEGATIVE_SCORE"]] = df["CLEANED_TEXT"].apply(lambda x: pd.Series(calculate_scores(x)))



In [ ]:
# Function to calculate positive and negative scores
def calculate_scores(tokens):
    pos_score = sum(1 for word in tokens if word in positive_words)
    neg_score = sum(1 for word in tokens if word in negative_words)
    return pos_score, neg_score

# Calculate Positive and Negative Scores
df[["POSITIVE_SCORE", "NEGATIVE_SCORE"]] = df["CLEANED_TEXT"].apply(lambda x: pd.Series(calculate_scores(x)))

In [ ]:
# Function to calculate average sentence length and percentage of complex words
def calculate_readability(text):
    if pd.notnull(text):  # Check if text is not null
        sentences = nltk.sent_tokenize(text)
        num_sentences = len(sentences)
        total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
        avg_sentence_length = total_words / num_sentences

        # Calculate the number of complex words (words with more than two syllables)
        complex_word_count = sum(1 for sentence in sentences for word in word_tokenize(sentence) if syllable_count(word) > 2)

        # Calculate the percentage of complex words
        percentage_complex_words = (complex_word_count / total_words) * 100 if total_words > 0 else 0

        return avg_sentence_length, percentage_complex_words
    else:
        return 0, 0  # Return 0 if the text is null

# Helper function to count syllables in a word
def syllable_count(word):
    word = word.lower()
    if len(word) <= 3:
        return 1
    return len(re.findall(r'[aeiouy]+', word))

# Calculate Average Sentence Length and Percentage of Complex Words
df[["AVG SENTENCE LENGTH", "PERCENTAGE OF COMPLEX WORDS"]] = df["TEXT"].apply(lambda x: pd.Series(calculate_readability(x)))


In [ ]:
# Function to calculate Fog Index
def calculate_fog_index(average_sentence_length, percentage_complex_words):
    return 0.4 * (average_sentence_length + percentage_complex_words)

# Calculate Fog Index
df["FOG INDEX"] = df[["AVG SENTENCE LENGTH", "PERCENTAGE OF COMPLEX WORDS"]].apply(lambda x: calculate_fog_index(x["AVG SENTENCE LENGTH"], x["PERCENTAGE OF COMPLEX WORDS"]), axis=1)



In [ ]:
# Function to count the number of syllables in a word
def count_syllables(word):
    word = word.lower()
    if word.endswith(("es", "ed")):
        word = word[:-2]
    vowels = "aeiouy"
    syllables = sum(word.count(vowel) for vowel in vowels)
    if word.startswith("y"):
        syllables += 1
    return max(1, syllables)




In [ ]:
# Function to calculate average number of words per sentence, complex word count, word count, syllable per word, personal pronouns, and average word length


def calculate_text_analysis(text):
    if pd.notnull(text):  # Check if text is not null
        words = word_tokenize(text)
        num_sentences = len(nltk.sent_tokenize(text))
        num_words = len(words)

        # Calculate average number of words per sentence
        avg_words_per_sentence = num_words / num_sentences if num_sentences > 0 else 0

        # Calculate complex word count (words with more than two syllables)
        complex_word_count = sum(1 for word in words if syllable_count(word) > 2)

        # Calculate word count
        word_count = len(words)

        # Calculate syllable per word
        syllable_per_word = sum(syllable_count(word) for word in words) / word_count if word_count > 0 else 0

        # Calculate personal pronouns count
        personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

        # Calculate average word length
        avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0

        # Return the results as a DataFrame with a single row
        return pd.DataFrame([[avg_words_per_sentence, complex_word_count, word_count, syllable_per_word, personal_pronouns, avg_word_length]], columns=["AVG NUMBER OF WORDS PER SENTENCE", "COMPLEX WORD COUNT", "WORD COUNT", "SYLLABLE PER WORD", "PERSONAL PRONOUNS", "AVG WORD LENGTH"])



In [ ]:
def calculate_text_analysis(text):
    if pd.notnull(text):  # Check if text is not null
        words = word_tokenize(text)
        num_sentences = len(nltk.sent_tokenize(text))
        num_words = len(words)

        # Calculate average number of words per sentence
        avg_words_per_sentence = num_words / num_sentences if num_sentences > 0 else 0

        # Calculate complex word count (words with more than two syllables)
        complex_word_count = sum(1 for word in words if syllable_count(word) > 2)

        # Calculate word count
        word_count = len(words)

        # Calculate syllable per word
        syllable_per_word = sum(syllable_count(word) for word in words) / word_count if word_count > 0 else 0

        # Calculate personal pronouns count
        personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

        # Calculate average word length
        avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0

        # Return the results as a Series
        return pd.Series({
            "AVG WORDS PER SENTENCE": avg_words_per_sentence,
            "COMPLEX WORD COUNT": complex_word_count,
            "WORD COUNT": word_count,
            "SYLLABLE PER WORD": syllable_per_word,
            "PERSONAL PRONOUNS": personal_pronouns,
            "AVG WORD LENGTH": avg_word_length
        })
    else:
        # Return a Series with default values (zeros) when text is NaN or None
        return pd.Series({
            "AVG WORDS PER SENTENCE": 0,
            "COMPLEX WORD COUNT": 0,
            "WORD COUNT": 0,
            "SYLLABLE PER WORD": 0,
            "PERSONAL PRONOUNS": 0,
            "AVG WORD LENGTH": 0
        })


In [ ]:
# Apply the function to the "TEXT" column
analysis_df = df["TEXT"].apply(calculate_text_analysis)

# Assign the calculated values to the original DataFrame 'df'
df[["AVG WORDS PER SENTENCE", "COMPLEX WORD COUNT", "WORD COUNT", "SYLLABLE PER WORD", "PERSONAL PRONOUNS", "AVG WORD LENGTH"]] = analysis_df


In [39]:
# Save the output to "Output Data Structure.xlsx" file
output_file = "Output Data Structure.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

In [43]:
of = pd.read_excel('Output Data Structure.xlsx')
of.head()

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,79,34,26.546667,26.117529,21.065678,26.546667,520,1991,1.890507,1,5.162230
1,38,https://insights.blackcoffer.com/what-if-the-c...,71,37,20.620253,16.574586,14.877936,20.620253,270,1629,1.610190,7,4.295273
2,39,https://insights.blackcoffer.com/what-jobs-wil...,73,36,22.607143,25.276461,19.153442,22.607143,480,1899,1.845708,3,4.901527
3,40,https://insights.blackcoffer.com/will-machine-...,79,27,19.384615,17.176871,14.624594,19.384615,303,1764,1.655896,18,4.422902
4,41,https://insights.blackcoffer.com/will-ai-repla...,65,25,25.315789,19.438669,17.901784,25.315789,374,1924,1.691268,16,4.585239


In [38]:
# Drop the "Title", "TEXT" and "Cleaned Text" columns from the original DataFrame "df"
df.drop(columns=["TITLE", "TEXT", "CLEANED_TEXT"], inplace=True)

# Save the modified DataFrame to the same Excel file (overwriting the original file)
df.to_excel("Output Data Structure.xlsx", index=False, engine="openpyxl")